# Hello
**We will seek answers to some questions asked to the data set.I hope this study can be useful. thank you to everyone already reading**

In [ ]:
import numpy as np 
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
data = pd.read_csv('../input/factors-affecting-campus-placement/Placement_Data_Full_Class.csv')
data.head()
# Read to data set

## **I think to check missing observations and others**

In [ ]:
data.isnull().any()

### **Missing observation in a variable**

- I will delete this because I couldn't fully control the story of the dataset.
- But the average can also be filled with solutions like estimation.

In [ ]:
CopyData = data.copy() # I copy the data set
CopyData.fillna(CopyData['salary'].mean(),inplace=True) # I fill in the missing values with the average.
#I will continue with all transactions with the copy data set.

In [ ]:
CopyData.isnull().any()
# Its Done

### **Let's check our variable information and descriptive statistics immediately**

In [ ]:
CopyData.describe().T
# Students in the classroom are on the same average.

### **Of course, we may have to check against the observer.**

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Box(y=CopyData['ssc_p'],name='Secondary Education percentage',boxpoints='all',jitter=0.3,marker_color='rgb(0,255,0)',line_color='rgb(50,205,50)'))
fig.add_trace(go.Box(y=CopyData['hsc_p'],name='Higher Secondary Education percentage',boxpoints='suspectedoutliers',jitter=0.3,marker_color='rgb(32,178,170)',line_color='rgb(102,205,170)'))
fig.add_trace(go.Box(y=CopyData['degree_p'],name='Degree Percentage',boxpoints='suspectedoutliers',jitter=0.3,marker_color='rgb(47,79,79)',line_color='rgb(0,128,128)'))
fig.add_trace(go.Box(y=CopyData['etest_p'],name='Employability test percentage',boxpoints='all',jitter=0.3,marker_color='rgb(127,255,212)',line_color='rgb(64,224,208)'))
fig.add_trace(go.Box(y=CopyData['mba_p'],name='MBA percentage',boxpoints='all',jitter=0.3,marker_color='rgb(0,191,255)',line_color='rgb(25,25,112)'))

# There's a little bit of contradiction,i don't want to touch this.
- Let's examine the connection between variables.
- I will do this with the help of heatmap.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
f, ax = plt.subplots(figsize=(10, 10))
sns.heatmap(data.corr(),ax=ax,annot=True,linewidths=.5,cmap="YlGnBu");

## **I guess there is no other variable that directly affects the Salary variable**
- Let's look at it from a different perspective.

In [ ]:
sns.pairplot(CopyData, kind  ="reg");

## **It sounds complicated but it expresses the connection of the variables better.**

**As a result, there is a little connection between the variables.I think we can set up a linear regression for the salary variable.**



->  Now let's look at selected specialties in secondary education.

In [ ]:
import plotly.express as px
colors = ['Gold','GoldenRod','Gray']
fig = go.Figure()
fig.add_trace(go.Bar(x=['Commerce','Science','Arts'],y=[113,91,11],marker_color=colors))
fig.show()

## ***Areas of expertise selected in this way***

- Let's take a look at the training board.

In [ ]:
colors = ['Crimson','LightPink']
fig = go.Figure()
fig.add_trace(go.Bar(x=['Others','Central'],y=[131,84],marker_color=colors))
fig.show()

## **Maybe we can make an inference that we need to look at the gender of those who choose specialization in secondary school**

In [ ]:
f, ax = plt.subplots(figsize=(8, 5))
sns.barplot(x='salary', y='gender',hue='hsc_s', data=CopyData,ax=ax,color='Crimson');

## **When we look here, we see that men receive a good salary from science and women from trade.**

## **Let's look at the salary effects of graduation specialties.**

In [ ]:
f, ax = plt.subplots(figsize=(10, 8))
sns.barplot(x='salary', y='gender',hue=CopyData['degree_t'], data=CopyData,ax=ax,color='BlueViolet');

## **If I comment on this,men continued to advance in science and technique.If we look at women, they turned to other branches.But as a general comment, their salaries increased on both sides**

## **Let's build a model like this,Let's check whether it is settled according to school scores.**

- Let's do this classification by random forest algorithm.


![](https://img.gazetevatan.com/vatanmediafile/Haber598x362/2020/02/07/ogm-sonuclari-belli-oldu-mu-ogm-sonuc-sorgulama-n-4395954.Jpeg)

## **First I want to make the following categorical variables 0 and 1**

In [ ]:
CopyDataX = CopyData.copy() # Let's copy whatever happens ;)
dms = pd.get_dummies(CopyData[['ssc_b','hsc_b','hsc_s','degree_t','workex','specialisation','status','gender']])
dms.head() # Let's clean this one too

In [ ]:
y = CopyData['status']
X_ = CopyData.drop(['ssc_b','hsc_b','hsc_s','degree_t','workex','specialisation','status','gender'],axis=1).astype('float64')
X_.head()

In [ ]:
X = pd.concat([X_,dms[['ssc_b_Central','hsc_b_Central','hsc_s_Arts','hsc_s_Commerce','hsc_s_Science','degree_t_Comm&Mgmt','degree_t_Others','degree_t_Sci&Tech','workex_Yes','specialisation_Mkt&Fin','status_Placed','gender_M']]],axis=1)
X.head()

## **I hope the effects of this separation will not affect us badly.**
- Let's do the separation of train and test sets.

In [ ]:
CopyDataT = X

In [ ]:
from sklearn.model_selection import train_test_split
y = CopyDataT['status_Placed']
x = CopyDataT.drop(['status_Placed'],axis=1)
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.30,random_state=42)
# We made our train and test distinctions.

# **Building a model**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RandomFC = RandomForestClassifier()
RandomFCmodel = RandomFC.fit(x_train,y_train)
print(RandomFCmodel)

In [ ]:
from sklearn.metrics import accuracy_score
y_prediction = RandomFCmodel.predict(x_test)
accuracy_score(y_test,y_prediction)
# Yes, we could have built a beautiful model.

## **Let's take a look at our real y and estimate y values.**

In [ ]:
Y_p = pd.DataFrame({
    'y_pred':y_prediction,
    'y_real':y_test
})
Y_p.head(10)
# Quite successful.

## **Let's try to improve our model a little more.**


# MODEL TUNİNG

In [ ]:
from sklearn.model_selection import GridSearchCV
RF_params = {
    'max_depth':[2,3,5,8,10],
    'max_features':[2,5,8],
    'n_estimators':[10,500,1000,2000],
    'min_samples_split':[2,5,10]
}
RF = RandomForestClassifier()
RF_model_cv = GridSearchCV(RF,RF_params,cv=10,n_jobs=-1,verbose=2)
RF_model_cv.fit(x_train,y_train)
# We are searching for hyperparameters.

In [ ]:
print('Best Parameter:' + str(RF_model_cv.best_params_))

## **Now let's create a new model using these parameters**

In [ ]:
RF_tuned = RandomForestClassifier(max_depth=RF_model_cv.best_params_['max_depth'],
                                  max_features=RF_model_cv.best_params_['max_features'],
                                 min_samples_split=RF_model_cv.best_params_['min_samples_split'],
                                 n_estimators=RF_model_cv.best_params_['n_estimators']).fit(x_train,y_train)

In [ ]:
y_predic_new = RF_tuned.predict(x_test)
accuracy_score(y_test,y_predic_new)
# Not much has changed, but we used the optimum parameters.

## **Let's take a look at which variables are more important in case of  being settled.**

- Importance of variables in classification.

In [ ]:
Importance = pd.DataFrame({'Importance':RF_tuned.feature_importances_*100},
                         index=x_train.columns)
Importance_sorted = Importance.sort_values(by='Importance',
                                          axis=0,
                                          ascending=True).plot(kind='barh',color='Coral');

## **Yes, when I look at the significance levels, I see that the scores affect but the salary variable affects too much**

- Experience, gender etc. It doesn't matter.


--> Let's try to set up a very linear regression model for the salary variable.

In [ ]:
MPData = pd.DataFrame({
    'ssc_p':CopyDataT['ssc_p'],
    'hsc_p':CopyDataT['hsc_p'],
    'degree_p':CopyDataT['degree_p'],
    'etest_p':CopyDataT['etest_p'],
    'mba_p':CopyDataT['mba_p'],
    'salary':CopyDataT['salary']
})
MPData.head()
# I don't think the rest will affect the rest except mba but I want to include them too.

In [ ]:
# We do train and test separation.
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split,cross_val_score,cross_val_predict
x_x = data.drop(['salary'],axis=1)
y_y = data['salary']
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.30,random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression
Linear = LinearRegression()
LinearModel = Linear.fit(x_train,y_train)
print('Linear Model İntercept' + str(LinearModel.intercept_))
print('Linear Model Coef' + str(LinearModel.coef_))

In [ ]:
rmse = np.sqrt(mean_squared_error(y_train,LinearModel.predict(x_train)))
print('Train Error:',rmse)
t_rmse = np.sqrt(mean_squared_error(y_test,LinearModel.predict(x_test)))
print('Test Error:',t_rmse)
print('Verified Error:',cross_val_score(LinearModel,x_train,y_train,cv=10,scoring='r2').mean())

# **Thank you for reading so much I can**